In [ ]:
import mne
import matplotlib
import numpy as np
%matplotlib inline
import PyQt5
%matplotlib qt
%gui qt5
import os
import os.path as op
import autoreject
from autoreject import AutoReject
from autoreject import get_rejection_threshold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook as tqdm

## Importing Raw Data

In [ ]:
overwrite = True
# eeg_path = r'E:\Ammara_UOL\Masters Thesis\Data\Final data'
eeg_path = r'D:\Master_thesis\Final_data'
subject = 'VR2FEM_S19'
data_path = os.path.join(eeg_path, subject , 'MainTask', 'EEG')
raw_fname = os.path.join(data_path,'%s' %subject)

In [ ]:
#reading eeg raw data
raws = []
if subject =='VR2FEM_S02':
    for files in os.listdir(data_path): 
        if files.endswith(".vhdr"):
            file_name = f'{data_path}\{files}'
            raws.append(mne.io.read_raw_brainvision(file_name, preload=True))
    raw = mne.concatenate_raws(raws)
else: 
    raw = mne.io.read_raw_brainvision(raw_fname + '.vhdr', preload=True)

### Plotting sensor location

In [ ]:
# Assigning Ocular electrodes
raw.set_channel_types(mapping={'AF7': 'eog', 'AF8': 'eog', 'FT9': 'eog', 'FT10': 'eog'})

# Setting montage from brainvision montage file
montage= mne.channels.read_custom_montage(r'D:\Master_thesis\RWKSP_Montages_BVEF_active\active electrodes\actiCAP_snap_CACS\CACS-64\CACS-64_REF.bvef')
raw.set_montage(montage)

# Plotting montage
raw.plot_sensors("topomap", show_names=True)

In [ ]:
raw.plot_psd(fmin=0, fmax=40, n_jobs=3)

In [ ]:
# reject vrey bad chans and not interpolate (spherical)
raw.info['bads'] = ['FC2', 'FC4']
print(raw.info['bads'])

picks = mne.pick_types(raw.info, exclude=['bads'])

In [ ]:
# raw.plot(order=picks, n_channels=len(picks))

## Filtering the data

In [ ]:
# Filtering: Band pass filter: 0.1-40Hz
sfreq = raw.info['sfreq']
l_cutoff = 0.1   
h_cutoff = 20   #try 20 Hz
iir_params = dict(order=6, ftype='butter')
raw_filtered = raw.copy().filter(l_freq=l_cutoff, h_freq=h_cutoff, picks=['eeg'], method='iir', iir_params = iir_params)

In [ ]:
raw_filtered.plot_psd(fmax=20)

### Extracting epochs

In [ ]:
#Creating event dictionary and extracting epochs


events_from_annot, event_dictionary  = mne.events_from_annotations(raw)

event_dict = {'face/mono/1/neutral':     111,
              'face/mono/1/happy':       112,
              'face/mono/1/angry':       113,
              'face/mono/1/surprised':   114,
              'face/mono/2/neutral':     121,
              'face/mono/2/happy':       122,
              'face/mono/2/angry':       123,
              'face/mono/2/surprised':   124,
              'face/mono/3/neutral':     131,
              'face/mono/3/happy':       132,
              'face/mono/3/angry':       133,
              'face/mono/3/surprised':   134,
              'face/stereo/1/neutral':   211,
              'face/stereo/1/happy':     212,
              'face/stereo/1/angry':     213,
              'face/stereo/1/surprised': 214,
              'face/stereo/2/neutral':   221,
              'face/stereo/2/happy':     222,
              'face/stereo/2/angry':     223,
              'face/stereo/2/surprised': 224,
              'face/stereo/3/neutral':   231,
              'face/stereo/3/happy':     232,
              'face/stereo/3/angry':     233,
              'face/stereo/3/surprised': 234,
             }
epochs = mne.Epochs(raw_filtered, events_from_annot, event_id=event_dict, tmin = -0.3, tmax= 1,baseline=None,  preload=True)
if subject == 'VR2FEM_S03': epochs  = epochs[:-4]     ## last 4 trials for subject 4 were duplicates so deleting from epochs
if subject == 'VR2FEM_S12': epochs  = epochs[3::]
if subject == 'VR2FEM_S16': epochs  = epochs.drop(np.where(epochs.events[:, 2]==133)[0][-1])  ## face/mono/3/angry is presented one more time than other emotions
if subject == 'VR2FEM_S18': epochs  = epochs[1::]
if subject == 'VR2FEM_S21': epochs  = epochs[2::]
# Plotting epochs with event markers 
# epoch_fig = epochs.plot(n_epochs=2, events=events_from_annot)

## Cleaning the atrifacts and ICA

Here first we highpass filter the data, then run autoreject and then using these bad epochs detected by it, to be feed to ICA algorithm, and finally run autoreject (local) again for cleaning data.

In [ ]:
epochs.average().plot()

In [ ]:
sfreq = raw.info['sfreq']
l_cutoff = 1   
h_cutoff = 20   #20 Hz
iir_params = dict(order=6, ftype='butter')
raw_highfilt = raw.copy().filter(l_freq=l_cutoff, h_freq=h_cutoff, picks=['eeg'], method='iir', iir_params = iir_params) #highpass filter on copy of the data to get rid of slow drifts

epochs_highfilt = mne.Epochs(raw_highfilt, events_from_annot, event_id=event_dict, tmin = -0.3, tmax= 1,baseline=None, preload=True)
if subject == 'VR2FEM_S03': epochs_highfilt  = epochs_highfilt[:-4]     ## last 4 trials for subject 4 were duplicates so deleting from epochs
if subject == 'VR2FEM_S12': epochs_highfilt  = epochs_highfilt[3::]
if subject == 'VR2FEM_S16': epochs_highfilt  = epochs_highfilt.drop(np.where(epochs_highfilt.events[:, 2]==133)[0][-1]) 
if subject == 'VR2FEM_S18': epochs_highfilt  = epochs_highfilt[1::] 
if subject == 'VR2FEM_S21': epochs_highfilt  = epochs_highfilt[2::] 

### Splitting test and training sets

In [ ]:
X = epochs_highfilt
event = epochs_highfilt.events[:,2] 
y = np.array([int(str(yy)) for yy in event])

n_splits = 5
train_index_list = []
test_index_list = []

random_state= int(input('Enter random seed number: '))
skf = StratifiedKFold(n_splits=n_splits,  shuffle=True, random_state= random_state)
for train_index, test_index in skf.split(X, y):
        train_index_list.append(train_index)
        test_index_list.append(test_index)

# ---->    Start next split from here

In [ ]:
data_split_set = int(input("Select split number train/test set: "))
n = data_split_set - 1
epochs_highfilt_train = epochs_highfilt[train_index_list[n]] 
epochs_highfilt_test = epochs_highfilt[test_index_list[n]]

In [ ]:
print(n)

In [ ]:
ar = autoreject.AutoReject(n_jobs=3)               
ar.fit(epochs_highfilt_train[:])   # make subset for test and training data, do estimation on the train epochs and apply the result to both train and test set

In [ ]:
epochs_highfilt_ar, reject_log = ar.transform(epochs_highfilt_train, return_log = True)

visualize bad epochs

In [ ]:
# epochs_highfilt_train[reject_log.bad_epochs].plot(scalings=dict(eeg=100e-6))

and visualize reject log.

In [ ]:
# reject_log.plot('horizontal')

**Supplying bad epoch log from Autoreject to ICA**

In [ ]:
ica = mne.preprocessing.ICA(method="picard")
ica.fit(epochs_highfilt_train[~reject_log.bad_epochs])    # make subset for test and training data, do estimation on the train epochs and apply the result to both train and test set

### Using an EOG channels to select ICA components automatically

In [ ]:
ica.exclude = []
eog_indices, eog_scores = ica.find_bads_eog(raw)               # find which ICs match the EOG pattern
ica.exclude = eog_indices

ica.exclude

In [ ]:
from mne_icalabel import label_components

# assuming you have a Raw and ICA instance previously fitted
ic_labels = label_components(epochs, ica, method='iclabel')
labels = ic_labels["labels"]
exclude_idx = [idx for idx, label in enumerate(labels) if label not in ["brain"]]
print(f"Excluding these ICA components: {exclude_idx}")

**Manual picking of components**

In [ ]:
# ica.plot_sources(epochs)

In [ ]:
ica.plot_components(range(0, 58), inst= epochs)

In [ ]:
# ica.plot_properties(epochs)


In [ ]:
ica.exclude

In [ ]:
epochs_ica = epochs.copy()   
ica.apply(epochs_ica)

In [ ]:
ica.plot_overlay(epochs.average(), exclude=ica.exclude)

**Plotting epochs after applying ICA**

In [ ]:
# epochs_ica.plot()

In [ ]:
epochs_ica.plot_psd(fmax=20)

**Applying baseline correction**

In [ ]:
epochs_ica.apply_baseline((-0.2, 0))

In [ ]:
epochs_ica.average().plot()

### Splitting test and training sets

In [ ]:
epochs_train_ica = epochs_ica[train_index_list[n]]
epochs_test_ica = epochs_ica[test_index_list[n]]

### Autoreject after ICA

In [ ]:
ar = autoreject.AutoReject(n_jobs=3)               
ar.fit(epochs_train_ica[:])

In [ ]:
epochs_train_clean, reject_log_train = ar.transform(epochs_train_ica, return_log=True)
epochs_test_clean, reject_log_test = ar.transform(epochs_test_ica, return_log=True)

In [ ]:
# epochs_train_ica[reject_log_train.bad_epochs].plot(scalings=dict(eeg=100e-6))
# reject_log_train.plot('horizontal')

In [ ]:
reject_log_train.plot_epochs(epochs_train_ica)

In [ ]:
epochs_train_clean.plot()

In [ ]:
epochs_test_clean.plot()

In [ ]:
epochs_train_clean.average().plot()

In [ ]:
epochs_test_clean.average().plot()

In [ ]:
epochs_train_clean

**Saving epochs**

In [ ]:
data_path_out = r'D:\Master_thesis'
dir_save= os.path.join(data_path_out, 'Pre-processed_data', subject)
epochs_train_name = os.path.join(f"{dir_save}\{subject}-preprocessed_train_{data_split_set}-epo.fif")
epochs_test_name = os.path.join(f"{dir_save}\{subject}-preprocessed_test_{data_split_set}-epo.fif")

if not op.exists(dir_save):
            os.makedirs(dir_save)
overwrite = True
epochs_train_clean.save(epochs_train_name, overwrite = overwrite)
epochs_test_clean.save(epochs_test_name, overwrite = overwrite)

if data_split_set>=5:
    print(f"Epochs for train/test for all splits saved")
else:
    print(f"Epochs for train/test split no. {data_split_set} saved, continue next iteration with split no. {data_split_set+1}")

